#Predict mosquitos dayswise (groupby date -.-)
because if we don't, we have data points that are exactly the same except that the numMosquitos is different, which HAS to fuck up the classifier.

## load and prepare data

In [160]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import cross_validation, preprocessing, metrics

In [161]:
df_train = pd.read_csv('aggregated_train_with_nummosquitos.csv')# from make_aggregated_with_Nummosquitos
#df_test = pd.read_csv('../test_filled_new.csv')
#orig = pd.read_csv('../test_filled_new.csv')
df_test = pd.read_csv('../test_filled_new.csv')

### encode species and trap, set unspecified to mix, drop strings

In [162]:
df_train['counter'] = 1

In [163]:
X = df_train.copy()


In [164]:
string_features = ['Address','Street','AddressNumberAndStreet','CodeSum']
species_encoder = preprocessing.LabelEncoder()
trap_encoder = preprocessing.LabelEncoder()
#X_dump = X[string_features]
df_dump = df_test[string_features]
#X_dump = X[string_features]

df_test.Species[df_test.Species == 'UNSPECIFIED CULEX'] = 'CULEX PIPIENS/RESTUANS'
#X.Species = species_encoder.fit_transform(X.Species)
y = df_train['NumMosquitos']
#df_test.Species = species_encoder.transform(df_test.Species)
#df_train.Species = species_encoder.transform(df_train.Species)

/home/thomas/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### groupby date, trap, species

either:

1. disregard species and only predict num mosquitos independently

2. predict number mosquitos per species (make new variable for each species or it won't fit in the dates!). In this case we also need to store information about in how many '50 sets' there was Wnvpresent.

In [165]:
groups = ['Date','Trap','Species']

In [166]:
tmp = X.groupby(groups)['NumMosquitos'].sum()


In [167]:
X_tmp = X.join(X.groupby(groups)['NumMosquitos'].sum(), on= groups,rsuffix='_counter')

In [168]:
X_tmp.NumMosquitos_counter.max()

2532

In [170]:
y = X_tmp.NumMosquitos_counter
y.max()

2532

now NumMosquiots_counter actually represents the number of mosquitos caught at a datapoint, so we can regress over it.

In [147]:
from sklearn.cross_validation import train_test_split, ShuffleSplit,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, SGDRegressor
import numpy as np

In [148]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import cross_validation, preprocessing, metrics
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [182]:

clf_list= [#RandomForestRegressor(n_estimators=100,n_jobs=-1),
           #RandomForestRegressor(n_estimators=400,n_jobs=-1),
           #RandomForestRegressor(n_estimators=1000,n_jobs=-1), 
           #Ridge(alpha = .01,normalize=True),
           #Ridge(alpha = .1,normalize=True),
           #Ridge(alpha = .1),
           #GradientBoostingRegressor(n_estimators = 100,max_depth=7),
           GradientBoostingRegressor(n_estimators = 100,max_depth=10),
           GradientBoostingRegressor(n_estimators = 100,max_depth=15,subsample = .7),#so far the best one, n_es = 400
           GradientBoostingRegressor(n_estimators = 1000,max_depth=15),
           GradientBoostingRegressor(n_estimators = 100,max_depth=20,subsample = .7)
           #KNeighborsRegressor(),
           #SGDRegressor()
           #GradientBoostingRegressor(n_estimators = 100,max_depth=4),
           #GradientBoostingRegressor(n_estimators = 100,max_depth=2)
      ]
random_state= 46


In [172]:
del X_tmp['NumMosquitos']

In [173]:
y = X_tmp.NumMosquitos_counter

In [174]:
del X_tmp['NumMosquitos_counter']

string_features = ['Date','Address','Street','AddressNumberAndStreet','CodeSum','Trap']

X = X.drop(string_features,axis=1)
df_test.Species[df_test.Species == 'UNSPECIFIED CULEX'] = 'CULEX PIPIENS/RESTUANS'
X.Species = species_encoder.fit_transform(X.Species)
#y = df_train['NumMosquitos']
df_test.Species = species_encoder.transform(df_test.Species)
df_train.Species = species_encoder.transform(df_train.Species)

In [176]:
y.max()

2532

In [183]:
X = scale(X)


err = []
r2 = []
#get lengths of y_arr -.-

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=.33, random_state= random_state)

y_arr = pd.DataFrame(y_test)
i = 1
#clf = clf_list[1]
for clf in clf_list:
#for i in np.random.randint(1,size=4):
    i += 1
    X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=.33, random_state= random_state)
    clf.fit(X_train,y_train)
    yhat = clf.predict(X_test)
    err.append( mean_squared_error(yhat,y_test))
    r2.append(r2_score(yhat,y_test))
    y_arr[i] = yhat
    

In [184]:
y_arr_mean = y_arr.drop(0,axis=1)
y_arr_mean = y_arr_mean.mean(axis=1)
y_arr_mean
r2_score(y_arr_mean,y_arr[0])

0.94351071807987263

In [185]:
r2

[0.94461017966290239,
 0.93408634464712315,
 0.94253866349789039,
 0.93751700742000632]

In [189]:
yhat.mean(),y_test.mean()

(100.93392403010652, 100.67580040380733)

In [190]:
err

[6640.4222545330631,
 7835.4273483826682,
 6957.1106058743626,
 7436.9597026691126]

In [191]:
sum((yhat-y_test)**2**1/2)

12891969.644576918

In [195]:
bla = pd.DataFrame(yhat,columns = ['hat'])
bla['orig'] = y_test

In [196]:
bla

,hat,orig
0,1.014467,1
1,27.703622,24
2,1.208408,1
3,51.712036,50
4,0.901477,1
5,301.193047,329
6,2.641549,2
7,24.126648,25
8,7.417122,8
9,49.708182,23
